![alt text](https://drive.google.com/uc?export=view&id=1UXScsVx_Wni_JuDdB8LeTnM6jsPfIwkW)

Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution prohibited.

# Sentiment Classification

### Dataset
- Dataset of 50,000 movie reviews from IMDB, labeled by sentiment positive (1) or negative (0)
- Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Command to import data
- `from tensorflow.keras.datasets import imdb`

In [1]:
pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4835 sha256=cd5dc89ea9366ee8dde081c2dd03dd692035a20611f5b11b41e7543ed9880ef6
  Stored in directory: C:\Users\CHANDRIMA CHATTARAJ\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built termcolor
  Found existing installation: h5py 2.9.0
    Uninstalling h5py-2.9.0:
      Successfully uninstalled h5py-2.9.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers

### Import the data (2 Marks)
- Use `imdb.load_data()` method
- Get train and test set
- Take 10000 most frequent words

In [5]:
from tensorflow.keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

### Pad each sentence to be of same length (2 Marks)
- Take maximum sequence length as 300

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
data = pad_sequences(data, maxlen=300)

### Print shape of features & labels (2 Marks)

In [13]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Categories: [0 1]
Number of unique words: 9999
Average Review length: 300.0
Standard Deviation: 0.0


Number of review, number of words in each review

In [16]:
print("Number of total reviews:",data.shape[0])

Number of total reviews: 50000


In [ ]:
#### Add your code here ####

Number of labels

In [18]:
print("Number of total labels:",targets.shape[0])

Number of total reviews: 50000


### Print value of any one feature and it's label (2 Marks)

Feature value

In [20]:
print("Number of unique words:", len(np.unique(np.hstack(data))))

Number of unique words: 9999


Label value

In [19]:
print("Unique labels:", np.unique(targets))

Unique labels: [0 1]


### Decode the feature value to get original sentence (2 Marks)

First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [22]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 

Now use the dictionary to get the original words from the encodings, for a particular sentence

In [23]:
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the st

Get the sentiment for the above sentence
- positive (1)
- negative (0)

In [28]:
if(targets[0] == 1): 
    print("Sentiment:Positive")
else:
    print("Sentiment:Positive")

Sentiment:Positive


In [29]:
def vectorize(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results
 
data = vectorize(data)
targets = np.array(targets).astype("float32")

In [30]:
data.shape

(50000, 10000)

In [31]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [102]:
print("Train Set size:",train_y.shape[0])
print("Test Set size:",test_y.shape[0])

Train Set size: 40000
Test Set size: 10000


### Define model (10 Marks)
- Define a Sequential Model
- Add Embedding layer
  - Embedding layer turns positive integers into dense vectors of fixed size
  - `tensorflow.keras` embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unique integer number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn LabelEncoder.
  - Size of the vocabulary will be 10000
  - Give dimension of the dense embedding as 100
  - Length of input sequences should be 300
- Add LSTM layer
  - Pass value in `return_sequences` as True
- Add a `TimeDistributed` layer with 100 Dense neurons
- Add Flatten layer
- Add Dense layer

In [114]:
# Input - Layer
model = models.Sequential()
model.add(layers.Dense(100, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

### Compile the model (2 Marks)
- Use Optimizer as Adam
- Use Binary Crossentropy as loss
- Use Accuracy as metrics

In [115]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

### Print model summary (2 Marks)

In [116]:
model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_110 (Dense)            (None, 100)               1000100   
_________________________________________________________________
dropout_46 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_111 (Dense)            (None, 100)               10100     
_________________________________________________________________
dropout_47 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_112 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_113 (Dense)            (None, 1)                 101       
Total params: 1,020,401
Trainable params: 1,020,401
Non-trainable params: 0
___________________________________________

### Fit the model (2 Marks)

In [117]:
results = model.fit(
 train_x, train_y,
 epochs= 20,
 batch_size = 500,
 validation_data = (test_x, test_y)
)

Epoch 1/20
80/80 [==============================] - 15s 184ms/step - loss: 0.3749 - accuracy: 0.8338 - val_loss: 0.2791 - val_accuracy: 0.8848
Epoch 2/20
80/80 [==============================] - 2s 31ms/step - loss: 0.2054 - accuracy: 0.9199 - val_loss: 0.2936 - val_accuracy: 0.8850
Epoch 3/20
80/80 [==============================] - 2s 24ms/step - loss: 0.1360 - accuracy: 0.9498 - val_loss: 0.3201 - val_accuracy: 0.8796
Epoch 4/20
80/80 [==============================] - 2s 26ms/step - loss: 0.0794 - accuracy: 0.9738 - val_loss: 0.4235 - val_accuracy: 0.8781
Epoch 5/20
80/80 [==============================] - 2s 28ms/step - loss: 0.0442 - accuracy: 0.9853 - val_loss: 0.4940 - val_accuracy: 0.8792
Epoch 6/20
80/80 [==============================] - 2s 25ms/step - loss: 0.0263 - accuracy: 0.9912 - val_loss: 0.5752 - val_accuracy: 0.8743
Epoch 7/20
80/80 [==============================] - 2s 25ms/step - loss: 0.0192 - accuracy: 0.9935 - val_loss: 0.6419 - val_accuracy: 0.8782
Epoch 8/20


### Evaluate model (2 Marks)

In [118]:
print(np.mean(results.history["val_accuracy"]))

0.8775649994611741
